# Build a CNN

We have discussed the different components of a convolutional neural network; now, we can bring them all together.
For this, we will return to the FashionMNIST dataset we saw [previously](./build-your-own.ipynb). 

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from IPython.utils import io

with io.capture_output() as captured:
    training_data = datasets.FashionMNIST(
        root="../data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    test_data = datasets.FashionMNIST(
        root="../data",
        train=False,
        download=True,
        transform=ToTensor(),
    )

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

With the data loaded, the next step is to create the model. 
This model consists of three convolutional layers, each using a ReLU activation function, and is followed by a maximum pooling. 
Following the final maximum pooling, the data is flattened to be passed to the linear, fully connected layers (of which there are four). 
The final fully connected layer produces 10 output classes, matching the 10 types of clothing in the dataset. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    """
    A convolutional neural network model for image classification.
    """
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=0)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 3 * 3, 250)  # Adjusted for 28x28 input after convolution and pooling
        self.fc2 = nn.Linear(250, 125)
        self.fc3 = nn.Linear(125, 60)
        self.fc4 = nn.Linear(60, 10)  # 10 classes for classification

    def forward(self, x):
        """
        Forward pass of the neural network.
        
        :param x: The input tensor
        :return: The output tensor
        """
        # Outputs: 32 × 26 × 26 → 32 × 13 × 13
        x = self.pool(F.relu(self.conv1(x)))  
        # Output: 64 × 11 × 11 → 64 × 5 × 5
        x = self.pool(F.relu(self.conv2(x)))  
        # Output: 64 × 3 × 3
        x = F.relu(self.conv3(x))             

        # Flattens for fully connected layers to (batch_size, 64 * 3 *3)
        x = torch.flatten(x, start_dim=1) 
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        # No activation, as CrossEntropyLoss in PyTorch expects raw logits
        # x = self.fc4(x)  

        return x

With the simple model built, we can create the object and print what the model looks like.

In [ ]:
model = CNNModel()

print(model)

Next, we define our loss function (here, we use cross-entropy loss as we are working with a classification problem) and the optimiser. 

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Everything is now ready to train the network for 10 epochs. 

In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train() 
    for epoch in range(epochs):
        running_loss = 0.0

        for images, labels in train_loader:
            optimizer.zero_grad()  
            outputs = model(images)  
            loss = loss_fn(outputs, labels)  
            loss.backward()  
            optimizer.step() 

            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

train_model(model, train_dataloader, loss_fn, optimizer, epochs=10)


Then, with the model trained, we can look at how well it handles the test data. 

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

model.eval()  # Set model to evaluation mode
f1_scores = []
accuracy_scores = []

with torch.no_grad():  # No need to compute gradients during evaluation
    for images, labels in test_dataloader:

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get class with highest probability

        f1_scores.append(f1_score(labels, predicted, average='macro'))
        accuracy_scores.append(accuracy_score(labels, predicted))

print(f"Average Test Accuracy: {np.mean(accuracy_scores):.2f}%")
print(f"Average Test F1 Score: {np.mean(f1_scores):.2f}")

The average accuracy and F1 score are high, indicating that even for this simple network, the model can classify the images better than our [simple linear network](./build-your-own.ipynb).